# Generadores e iteradores

## Iteradores

Un [iterador](https://en.wikipedia.org/wiki/Iterator) es un objeto que permite recorrer los elementos de un contenedor, generalmente listas.

En python, se crear iteradores utilizando el método [iter](https://docs.python.org/3/library/functions.html#iter) sobre un contenedor.


In [59]:
l = [i for i in range(4)]
>>> [0, 1, 2, 3,]
it = iter(l)
it

En python, un generador debe soportar se invocado mediante la función `next` ante lo cual, retornará el siguiente valor de la colección que esté iterando

In [60]:
next(it)
>>> 0
next(it)
>>> 1
next(it)
>>> 2
next(it)

3

Si volvermos a invocar `next` una vez que el iterador se ha agotado, este lanzará la excepción `StopIteration`

In [27]:
next(it)

StopIteration: 

Vistos así, no parecen muy interesantes.

Pero podriamos pensar el funcionamiento internos del *for loop*, por ejemplo, de la siguiente manera: internamente cuando recibe una lista (o cualquier tipo de colección) genera un iterador de la misma mediante el método `iter`, y luego va obteniendo los elementes de la misma mediante `next`, hasta que recibe la señal de `StopIteration`.

Solo para ver como sería la implementación, crearemos la siguiente función (no es algo que vayamos a hacer al programar):


In [40]:
def mi_for( seq, f=print ):
    it = iter(seq)
    try:
        while True:
            f(next(it))
    except StopIteration:
        return

mi_for([1,3,4])

1
3
4


Otro ejemplo de como podría usarse

In [42]:
mi_for( range(5), f=lambda x: print(x*x))

0
1
4
9
16


En sí, en python, los generadores, rara vez se itilizan directamente, sino que generalmente se usan a través de un `for`, u otras funciones que están preparadas para utilizarlos. Sin embargo, uno a veces podría valerse de los mismos. 

Supongamos que tenemos dos listas, y queremos reagrupar los elementes de manera que por cada dos de la primera, haya uno de la segunda. Esto puede hacerse con un simple `while`, manejando numericamente los indices, pero probemos con un iterador:

In [100]:
l1 = list(range(10))
>>> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
l2 = list(map(chr,range(ord('a'),ord('f'))))
>>> ['a', 'b', 'c', 'd', 'e']


['a', 'b', 'c', 'd', 'e']

Modo tradicional

In [107]:
i = 0
hasta = min(len(l1),len(l2))
new_list = []
while i < hasta:
    new_list.append( 
        ( l1[i*2], l1[(i*2)+1], l2[i]  )
    )
    i += 1
new_list

[(0, 1, 'a'), (2, 3, 'b'), (4, 5, 'c'), (6, 7, 'd'), (8, 9, 'e')]

Utilizando iteradores

In [104]:
it1, it2 = iter(l1), iter(l2)
new_list = []
try: 
    while True:
        new_list.append( 
            (next(it1), next(it1), next(it2))
        )
except StopIteration: pass
new_list

[(0, 1, 'a'), (2, 3, 'b'), (4, 5, 'c'), (6, 7, 'd'), (8, 9, 'e')]

Por supuesto, esto puede excribirse de una forma más simple, utilizando la función `zip`.

In [106]:
it1 = iter(l1)
list( zip( it1, it1, iter(l2) ))

[(0, 1, 'a'), (2, 3, 'b'), (4, 5, 'c'), (6, 7, 'd'), (8, 9, 'e')]

En el ejemplo anterior, utilizamos `list` para generar una lista en sí, ya que la función `zip` de por sí, lo que devuelve es un iterador

In [123]:
it1 = iter(l1)
zit = zip( it1, it1, iter(l2) )
zit

In [124]:
next(zit)

(0, 1, 'a')

In [125]:
next(zit)

(2, 3, 'b')

Los constructores de secuencias, en python, están preparados para recibir iteradores.

Los iteradores, al ser invocados por la función `iter`, deben retornarse a sí mismos, de esta forma, pueden ser utilizados en como iterables.

In [126]:
it = iter(zit)
next(it)

(4, 5, 'c')

In [127]:
list(it)

[(6, 7, 'd'), (8, 9, 'e')]

## Generadores

Los [generadores](https://en.wikipedia.org/wiki/Generator_%28computer_programming%29) son un tipo especial de rutinas que pueden ser iteradas. A diferencia de una rutina, en donde la misma se ejecuta completamente y devuelve un valor, el generador es un caso especial de [corutina](https://en.wikipedia.org/wiki/Coroutine), esto es, una rutina con multiples puntos de acceso, en los cuales se suspende y continua la ejecución iterativamente, y en casa uno de estos accesos, o llamas, produce un objeto o valor.

En efecto, dicho así es dificil de entender. Pero la idea básica es que a diferencia de las subrutinas que devuelven un único valor, al igual que los iteradores, este tipo de corutinas van _produciendo_ valores a medida que son iterados.

En ptyhon, los generadores se definen de manera similar a una función, utilizando la palabra clave _def_. Lo que los identifica como generadores es que en algún lugar de su implementación, utilizan la palabra clave _yield_ para indicar cual es el valor (u objeto) que producirán.

Veamos un simple ejemplo (no es lo más común esto, pero sirve bien para introducir el concepto).

In [165]:
def config_files():
    
    print('antes de user')
    yield 'config/user.conf'
    
    print('antes de base')
    yield 'config/base.conf'
     
    print('antes de default')
    yield 'config/default.conf'
     
    print('terminamos')
    return 
        
config_files

<function __main__.config_files>

In [166]:
g = config_files()
g

<generator object config_files at 0x7fbcb91610d8>

In [167]:
next(g)

antes de user


'config/user.conf'

In [168]:
next(g)

antes de base


'config/base.conf'

In [169]:
next(g)

antes de default


'config/default.conf'

In [170]:
next(g)

terminamos


StopIteration: 

Como vemos, esta función devuelve un iterador, 
y cada vez que se lo invoca mediante `next`, este ejecuta el código del generador hasta encontrar un `yield` y retorna el valor producido.

De manera más simple, podría ejecutarse de la siguiente manera:

In [172]:
for f in config_files():
    print('yield -> ', f)

antes de user
yield ->  config/user.conf
antes de base
yield ->  config/base.conf
antes de default
yield ->  config/default.conf
terminamos


Veamos, por ejemplo, como sería la función `enumerate`, en un lenguaje que no soporta corutinas, o generadores:

In [270]:
def enumerate_func(seq):
    ret_list = []
    i = 0
    for e in seq:
        ret_list.append( (i,e) )
        i+=1
    return ret_list



Luego, como sería implementada con un generador

In [273]:
def enumerate_gen(seq):
    i = 0
    for e in seq:
        yield (i, e)
        i+=1

Ahora veamos com responde cada una

In [300]:
import sys
from builtins import enumerate

lista = tuple( ( str(i), str(i*i), i ) for i in range(1000000) )
print("Una lista de {} KB".format( round(sys.getsizeof( lista ) / 1024) ))

def test_func():
    for e in enumerate_func( lista ):
        pass
    return 1

def test_gen():
    for e in enumerate_gen( lista ):
        pass
    return 1

def test_builtin():
    for e in enumerate( lista ):
        pass
    return 1

%timeit test_func()
%timeit test_gen()
%timeit test_builtin()

Una lista de 7813 KB
1 loops, best of 3: 208 ms per loop
10 loops, best of 3: 124 ms per loop
10 loops, best of 3: 42.2 ms per loop


Sin embargo, esto no es todo. En ambos casos, los loops se corrieron de forma completa. Supongamos ahora, que solo necesitamos buscar un elemento en la lista, y saber su indice (esto, por supuesto, puede hacerse de otra manera, pero sigamos con el ejemplo):

In [303]:
def find_func( seq, x):
    for i,e in enumerate_func(seq):
        if e[0] == str(x):
            return i
    return None

%timeit find_func(lista, 100)   

1 loops, best of 3: 196 ms per loop


In [304]:
def find_gen( seq, x):
    for i,e in enumerate_gen(seq):
        if e[0] == str(x):
            return i
    return None

%timeit find_gen(lista, 100)

10000 loops, best of 3: 39.1 µs per loop


islice